<a href="https://colab.research.google.com/github/alirezash97/Galaxy-Classification/blob/master/Galaxy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# !pip install kaggle
# !mkdir .kaggle
# !ls -a
# import json
# token = {"username":"alirezashafaei97","key":"9cb262aa0c5658ffc4eb45857c41903c"}
# with open('/content/.kaggle/kaggle.json', 'w') as file:
#     json.dump(token, file)
# !mkdir ~/.kaggle
# !cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
# !kaggle config set -n path -v{/content}
# !chmod 600 /root/.kaggle/kaggle.json
# !kaggle datasets list
# !kaggle datasets list -s galaxy-zoo
# !kaggle datasets download -d zhuangjw/galaxy-zoo-cleaned -p /content
# !unzip /content/galaxy-zoo-cleaned.zip
# !pip install netCDF4

In [0]:

import numpy as np
import tensorflow as tf
import pandas as pd
from PIL import Image
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras import layers
from tensorflow.keras import Model
import cv2
from tqdm import tqdm
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot
from numpy import expand_dims
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import keras
import tensorflow.keras
from keras import backend as K


In [0]:
from netCDF4 import Dataset
f = Dataset('/content/galaxy_train.nc')
images = f.variables['image_train']
labels = f.variables['label_train']
t = Dataset('/content/galaxy_test.nc')
images_test = t.variables['image_test']


In [85]:
print(images.shape)
print(images_test.shape)

(61578, 64, 64, 3)
(79975, 64, 64, 3)


In [0]:
(train_images, train_labels) = images[:60000], labels[:60000]
(test_images, test_labels) = images[60000:61578], labels[60000:61578]


In [0]:
train_images = train_images/255
test_images = test_images/255

In [0]:


def identity_block(X, f, filters, stage, block):


  conv_name_base = 'res' + str(stage) + block + '_branch'
  bn_name_base = 'bn' + str(stage) + block + '_branch'

  F1, F2, F3 = filters 
  X_shortcut = X

  # first component
  X = tf.keras.layers.Conv2D(filters = F1, kernel_size=(1, 1), strides=(1, 1),
            padding = 'valid', name = conv_name_base + '2a', kernel_initializer ='random_uniform')(X)
  X = tf.keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
  X = tf.keras.layers.Activation('relu')(X) 

  # second component
  X = tf.keras.layers.Conv2D(filters = F2, kernel_size=(f, f), strides=(1, 1),
            padding = 'same', name = conv_name_base + '2b',kernel_initializer ='random_uniform')(X)
  X = tf.keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
  X = tf.keras.layers.Activation('relu')(X) 

  # Third component
  X = tf.keras.layers.Conv2D(filters = F3, kernel_size=(1, 1), strides=(1, 1),
            padding = 'valid', name = conv_name_base + '2c',kernel_initializer ='random_uniform')(X)
  X = tf.keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)
  X = tf.keras.layers.Activation('relu')(X) 

  # Final step 
  X = tf.keras.layers.Add()([X, X_shortcut])
  X = tf.keras.layers.Activation('relu')(X)

  return X



In [0]:
def convolutional_block(X, f, filters, stage, block, s):
  
  
  conv_name_base = 'res' + str(stage) + block + '_branch'
  bn_name_base = 'bn' + str(stage) + block + '_branch'

  F1, F2, F3 = filters 
  X_shortcut = X

  # first component
  X = tf.keras.layers.Conv2D(filters = F1, kernel_size=(1, 1), strides=(s, s),
            padding = 'valid', name = conv_name_base + '2a', kernel_initializer ='random_uniform')(X)
  X = tf.keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
  X = tf.keras.layers.Activation('relu')(X) 

  # second component
  X = tf.keras.layers.Conv2D(filters = F2, kernel_size=(f, f), strides=(1, 1),
            padding = 'same', name = conv_name_base + '2b', kernel_initializer ='random_uniform')(X)
  X = tf.keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
  X = tf.keras.layers.Activation('relu')(X) 

  # Third component
  X = tf.keras.layers.Conv2D(filters = F3, kernel_size=(1, 1), strides=(1, 1),
            padding = 'valid', name = conv_name_base + '2c', kernel_initializer ='random_uniform')(X)
  X = tf.keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)
  X = tf.keras.layers.Activation('relu')(X) 

  ###### shortcut path ######
  X_shortcut = tf.keras.layers.Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid',
                      name = conv_name_base + '1', kernel_initializer ='random_uniform')(X_shortcut)
  X_shortcut = tf.keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)


  # Final step 
  X = tf.keras.layers.Add()([X, X_shortcut])
  X = tf.keras.layers.Activation('relu')(X)

  return X




In [0]:
def ResNet50(input_shape, classes):
    
    # Define the input as a tensor with shape input_shape
    X_input = tf.keras.Input(input_shape)

    
    # Zero-Padding
    X = tf.keras.layers.ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = tf.keras.layers.Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = 'random_uniform')(X)
    X = tf.keras.layers.BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = tf.keras.layers.Activation('relu')(X)
    X = tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3
    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL
    X = tf.keras.layers.AveragePooling2D(pool_size=(2,2), padding='same')(X)

    # Output layer
    X = tf.keras.layers.Flatten()(X)
    X = tf.keras.layers.Dense(classes, activation='sigmoid', name='fc' + str(classes),kernel_initializer ='random_uniform')(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [0]:

model = ResNet50(input_shape = (64, 64, 3), classes = 37)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [105]:
model.fit(train_images, train_labels, epochs = 3, batch_size = 64)

Epoch 1/3
285/938 [========>.....................] - ETA: 42:53 - loss: 14.8709 - accuracy: 0.5639

KeyboardInterrupt: ignored

In [0]:
datagen = tf.keras.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)
datagen.fit(train_images)

In [0]:

history = model.fit_generator(datagen.flow(train_images, train_labels, batch_size=256),
                    steps_per_epoch=len(train_images) / 256, epochs=8)

# history = model.fit(train_images, train_labels, epochs=10, batch_size=128, 
#                     validation_data=(test_images, test_labels))

Epoch 1/8
 80/240 [========>.....................] - ETA: 7:50 - loss: 0.2406 - acc: 0.2533

KeyboardInterrupt: ignored

In [0]:
# (test_images, test_labels) = images[40000:45000], labels[40000:45000]


In [0]:
x = test_images[2:3]
y = test_labels[2:3]

print(y)

In [0]:
y_hat = model.predict(test_images)
# z = y - y_hat
# print(y)
print(y_hat.shape)
# print(z)

(5000, 37)


In [0]:
model.evaluate(test_images, test_labels)

5000/5000 [==============================] - 9s 2ms/sample - loss: 16.9797 - acc: 0.6048


[16.979708624267577, 0.6048]